In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost
import sklearn
import seaborn as sns
%matplotlib inline
data_path = '/Users/dirlt/.kaggle/competitions/bike-sharing-demand/'
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import sklearn.metrics
from sklearn.base import BaseEstimator, RegressorMixin
import skopt
from collections import defaultdict
from keras.models import Model
from sklearn import preprocessing

links:
- https://www.kaggle.com/miteshyadav/comprehensive-eda-with-xgboost-top-10-percentile/notebook
- https://www.kaggle.com/viveksrinivasan/eda-ensemble-model-top-10-percentile

# Load Data

In [42]:
df = pd.read_csv('mytrain.csv')
test_df = pd.read_csv('mytest.csv')
def norm(df, test_df, field):
    scaler = preprocessing.RobustScaler()
    df[field] = scaler.fit_transform(df[field].values.reshape((-1, 1))).ravel()
    test_df[field] = scaler.transform(test_df[field].values.reshape((-1, 1))).ravel()
norm(df, test_df, 'temp')
norm(df, test_df, 'windspeed')
X, y = df.drop(['casual', 'registered', 'count'], axis = 1), np.log1p(df[['casual', 'registered', 'count']])

In [45]:
def input_fn(x, casual = True):
    drop_fields = ['dt_day', 'dt_hour', 'season', 'weather', 'dt_year', 'dt_month', 'dt_weekday', 'atemp']
    if 'datetime' in x.columns:
        drop_fields.append('datetime')
    return x.drop(drop_fields, axis = 1)

def make_cv(X,n = 2):
    for i in range(n):
        days = [x for x in [18-i, 19-i]]
        train_idx = X[X['dt_day'].apply(lambda x: x not in days)].index
        test_idx = X[X['dt_day'].apply(lambda x: x in days)].index
        yield train_idx, test_idx

def rmse(x, y):
    return mean_squared_error(x, y) ** 0.5

def print_features(names, values, thres = 0.01):
    fts = list(zip(names, values))
    fts.sort(key = lambda x: -x[1])
    ns = []
    for idx, (name, value) in enumerate(fts):
        if value < thres: break
        print('- {} {:.2f}'.format(name, value))
        ns.append(name)
    print(format(','.join(ns)))

class MyEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        if 'ma' in kwargs:
            self.ma = kwargs['ma']
            del kwargs['ma']
        if 'mb' in kwargs:
            self.mb = kwargs['mb']
            del kwargs['mb']
        kwargs['init'] = True
        self.set_params(**kwargs)
        
    def fit(self, X, y):
        input_a = input_fn(X, casual=True)
        input_b = input_fn(X, casual=False)
        self.ma.fit(input_a, y['casual'])
        self.mb.fit(input_b, y['registered'])
        self.ca = input_a.columns
        self.cb = input_b.columns
        
    def predict(self, X, n = None):
        if n:
            ma_est = self.ma.estimators_
            mb_est = self.mb.estimators_
            self.ma.estimators_ = ma_est[:n]
            self.mb.estimators_ = mb_est[:n]
        ya = self.ma.predict(input_fn(X, casual=True))
        yb = self.mb.predict(input_fn(X, casual=False))
        y = np.log1p(np.expm1(ya) + np.expm1(yb))
        if n:
            self.ma.estimators_ = ma_est
            self.mb.estimators_ = mb_est
        return y
    
    def score(self, X, y , n = None):
        y2 = self.predict(X, n = n)
        return -rmse(y['count'], y2)
        
    def set_params(self, **params):
        pa = {}
        pb = {}
        for k in params:
            if k.startswith('a_'):
                pa[k[2:]] = params[k]
            elif k.startswith('b_'):
                pb[k[2:]] = params[k]
            elif k == 'n_estimators':
                # 让两个回归器共享n
                pa[k] = params[k]
                pb[k] = params[k]
            else:
                pass
        if 'init' not in params:
            #print(pa, pb)
            pass
        self.ma.set_params(**pa)
        self.mb.set_params(**pb)
        return self
        
    def get_params(self, deep = True):
        pa = self.ma.get_params(deep)
        pb = self.ma.get_params(deep)
        p = {}
        for k in pa:
            p['a_' + k] = pa[k]
        for k in pb:
            p['b_' + k] = pb[k]
        p['ma'] = self.ma
        p['mb'] = self.mb
        return p
    
    def print_features(self, thres = 0.005):
        ca = self.ca
        cb = self.cb
        print('=====casual=====')
        print_features(ca, self.ma.feature_importances_, thres)
        print('=====registered=====')
        print_features(cb, self.mb.feature_importances_, thres)

In [53]:
input_X = input_fn(X)
input_X.head(10)

,holiday,workingday,temp,humidity,windspeed,windspeed_0,humidity_0,humidity_1,humidity_2,humidity_3,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,0,-0.866667,81,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,-0.933333,80,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,-0.933333,80,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,-0.866667,75,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,-0.866667,75,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,-0.866667,75,-0.699732,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,-0.933333,80,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,-1.000000,86,-1.300268,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,-0.866667,75,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,-0.600000,76,-1.300268,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def skopt_min_fn(model, X, y, params, n_calls = 10, n_cv = 2):
    pitems = list(params.items())
    pkeys = [x[0] for x in pitems]
    pvalues = [x[1] for x in pitems]
    history = {}
    def _f(_params):
        my_params = dict(zip(pkeys, _params))
        print('use params = {}'.format(my_params))
        k = tuple(_params)
        if k in history:
            return history[k]
        model.set_params(**my_params)
        cv = make_cv(X, n_cv)
        scores = []
        for train_idx, test_idx in cv:
            model.fit(X.loc[train_idx], y.loc[train_idx])
            s = model.score(X.loc[test_idx], y.loc[test_idx])
            scores.append(s)
        s = -np.mean(scores)
        print('score = {}'.format(s))
        history[k] = s
        return s
    
    res = skopt.forest_minimize(func = _f, dimensions = pvalues, n_calls = n_calls)
    res.best_params = dict(zip(pkeys, res.x))
    res.best_score = res.fun
    return res

In [6]:
def select_n(model, X, y, ns, n_cv = 4):
    scores = defaultdict(list)
    scores2 = dict()
    cv = make_cv(X, n_cv)
    print('set n_estimators = {}'.format(max(ns)))
    model.set_params(**{'n_estimators': max(ns)})
    for idx, (train_idx, test_idx) in enumerate(cv):
        print('doing fit cv = {} ...'.format(idx))
        model.fit(X.loc[train_idx], y.loc[train_idx])
        for n in ns:
            s = model.score(X.loc[test_idx], y.loc[test_idx], n = n)
            scores[n].append(s)
    for n in ns:
        scores2[n] = np.mean(scores[n])
    tmp = list(scores2.items())
    tmp.sort(key = lambda x: -x[1])
    return scores2, tmp

# NN Model

In [ ]:
%%time
print('cv for rf model')
rf0 = RandomForestRegressor(n_estimators=200, random_state = 42, verbose=0, n_jobs=4)
rf1 = RandomForestRegressor(n_estimators=200, random_state = 42, verbose=0, n_jobs=4)
rf = MyEstimator(ma = rf0, mb = rf1)

In [ ]:
# %%time
# params = {'a_min_samples_split': [3, 10], 'b_min_samples_split': [3, 10]}
# rf_opt = skopt_min_fn(rf, X, y, params, n_calls = 20, n_cv = 4)
# print(rf_opt.best_params, rf_opt.best_score)

In [ ]:
# params = {'a_min_samples_split': [8,9,10,11,12], 'b_min_samples_split': [4,5,6,7,8]}
# rf_cv = GridSearchCV(rf, params, cv = make_cv(X,2), n_jobs = 1, verbose = 1)
# rf_cv.fit(X, y)
# print(rf_cv.best_score_, rf_cv.best_params_)

In [ ]:
# %%time
# rf.set_params(**{'a_min_samples_split': 10, 'b_min_samples_split': 7})
# rf_scores = select_n(rf, X, y, ns = range(100, 2000, 100), n_cv = 4)
# print(rf_scores[1][:5])

In [ ]:
%%time
rf_best_params = {'a_min_samples_split': 10, 'b_min_samples_split': 7, 'n_estimators': 1700}
rf.set_params(**rf_best_params)
rf.fit(X, y)
output_y = rf.predict(test_df)
output = np.round(np.expm1(output_y)).astype(int)
output[output < 0] = 0
df_output = pd.DataFrame({'datetime': test_df['datetime'], 'count': output}, columns=('datetime', 'count'))
df_output['count'] = df_output['count'].astype(int)
df_output.to_csv('submission-rf.csv', index = False)

In [ ]:
!kaggle competitions submit -c bike-sharing-demand -f submission-rf.csv -m 'ms=(10,7), n = 1700'